In [ ]:
fname = "Into the Depths Super Mario World - Part 4.txt"
start_line = 'EVENT_OBJECT_NAMECHANGE on class=AXVirtualView role=ROLE_SYSTEM_STATICTEXT name="'
start_line_len = len(start_line)
end_line = '"\n'

In [ ]:
import string
import Levenshtein

punctuation_trans = str.maketrans('', '', string.punctuation)
MAX_LEVENSTEIN = 16

with open(fname, "rt") as f:
    lines = f.readlines()
    
subtitles = [""] * 16

def clean_string(s):
    return s.translate(punctuation_trans).strip().lower()

def find_max_overlap(str1, str2):
    source_str1 = str1
    source_str2 = str2
    str1 = clean_string(str1)
    str2 = clean_string(str2)
    
    MIN_OVERLAP = 2
    max_overlap = 0
    overlap_str = ""
    for i in range(len(str1)):
        suffix = str1[i:]
        prefix = str2[:len(suffix)]
        if suffix == prefix:
            max_overlap = len(suffix)
            overlap_str = suffix
    return source_str1 + str2[max_overlap:] if max_overlap >= MIN_OVERLAP else None
    
BACKTRACK = 16

for line in lines:
    start_index = line.find(start_line)
    if start_index != -1:
        end_index = line.rfind(end_line)
        if end_index != -1:
            current_line = line[start_line_len:end_index]
            curLen = len(subtitles)
            lineLen = len(current_line)
            replaced = False
            for x in range(BACKTRACK):
                prevLine = subtitles[-BACKTRACK+x]
                prevLineLen = len(prevLine)
                if lineLen <= MAX_LEVENSTEIN:
                    if current_line[:prevLineLen] == prevLine:
                        subtitles[-BACKTRACK+x] = current_line
                        replaced = True
                        break 
                else:
                    if Levenshtein.distance(current_line[:prevLineLen], prevLine) < MAX_LEVENSTEIN:
                        subtitles[-BACKTRACK+x] = current_line
                        replaced = True
                        break
            if not replaced:
                subtitles.append(current_line)
    
#clean forward repeats
subtitles_filtered = subtitles[:BACKTRACK]
curLen = len(subtitles)
for x in range(curLen-BACKTRACK):
    foundSubstr = False
    for y in range(1, BACKTRACK):
        if subtitles[x+BACKTRACK-y].find(subtitles[x+BACKTRACK]) != -1:
            foundSubstr = True
            break
    if not foundSubstr:
        subtitles_filtered.append(subtitles[x+BACKTRACK])

In [ ]:
print(f"Length: {len(subtitles_filtered)}")
for x in range(len(subtitles_filtered)):
    print(f"{x}: {subtitles_filtered[x]}")